In [27]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import sys
import os
from scipy import stats

import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader


sys.path.insert(0, '/home/riccardo/Desktop/OcclusionInference/Architectures')

from models.BLT_models import multi_VAE
from data_loaders.dataset_sup import MyDataset_encoder, MyDataset_decoder
from models.Lin_model import Lin_model 

from solvers.utils_mod import DataGather, get_accuracy
from solvers.losses import supervised_encoder_loss


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
test = Lin_model(36,36)
print(test)

Lin_model(
  (layer1): Linear(in_features=36, out_features=36, bias=True)
)


In [37]:
train_image_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/train/orig/"
train_target_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/digts.csv"
gnrl_image_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/gnrl/orig/"
gnrl_target_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/digts_gnrl.csv"

dset = MyDataset_encoder
train_data_size = len(os.listdir(train_image_paths))

encoder_target_type = 'depth_ordered_one_hot_xy'

train_data = dset(train_image_paths,train_target_paths, 32, encoder_target_type, 'train',train_data_size )

train_loader = DataLoader(train_data,
                            batch_size=100,
                            shuffle=False,
                            num_workers=8,
                            pin_memory=True,
                            drop_last=False)

# gnrl_data = dset(gnrl_image_paths,gnrl_target_paths, 32, encoder_target_type, 'gnrl',train_data_size )
# gnrl_loader = DataLoader(gnrl_data,
#                             batch_size=100,
#                             shuffle=False,
#                             num_workers=8,
#                             pin_memory=True,
#                             drop_last=False)



Sorting train image files


In [38]:
net_1 = multi_VAE('BLT','BLT', 24, 0 ,32, 1, 4, False, 4, 1, False)
file_path = '/home/riccardo/Desktop/Experiments/AE/Unfrozen/2_digts/BLT_BLT_depth_zdim24_2/main/last'
checkpoint = torch.load(file_path) 
net_1.load_state_dict(checkpoint['model_states']['net'])


using BLT encoder BLT decoder


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [41]:

n_digits = 2

z_code = 24
lr = 1e-3
beta1= 0.9
beta2= 0.999
max_epoch = 10


In [43]:

if encoder_target_type == 'joint':
    z_out = 10
elif encoder_target_type == 'black_white':
    z_out = 20
elif encoder_target_type == 'depth_black_white':
    z_out = 21
elif encoder_target_type == 'depth_black_white_xy_xy':
    z_out = 25
elif encoder_target_type== "depth_ordered_one_hot":
    if n_digits == 2:
        z_out = 20
    elif self.n_digits ==3:
        z_out = 30
elif encoder_target_type== "depth_ordered_one_hot_xy":
    if n_digits == 2:
        z_out = 24
    elif n_digits ==3:
        z_out = 36
    
lin_net = Lin_model(z_code, z_out)
optim_2 = optim.Adam(lin_net.parameters(), lr=lr, betas=(beta1, beta2))    

if torch.cuda.device_count()>1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    lin_net = nn.DataParallel(lin_net)
lin_net = lin_net.to(device) 
print(lin_net)

iters_per_epoch = len(train_loader)
print(iters_per_epoch, 'iters per epoch')
max_iter = max_epoch*iters_per_epoch
batch_size = train_loader.batch_size

count = 0
out = False

count_list = []
Tot_loss_list = []
back_loss_list = []
mid_loss_list = []
front_loss_list = []
xy_loss_list = []
back_accuracy_list = []
mid_accuracy_list = []
front_accuracy_list = []

while not out:
    for sample in train_loader:
        if count%500==0:
            print(count)

        x = sample['x'].to(device)
        y = sample['y'].to(device)
    
        net_1 = net_1.to(device)
        with torch.no_grad():
            output_list = net_1._encode(x)
            
            output = output_list[-1]
            output = output[:, :z_out]
        
        final_out = lin_net(output)
        
  
        loss_list = supervised_encoder_loss(final_out, y, n_digits, encoder_target_type)
        loss = loss_list[0]
        
#         l2 = 0
#         for p in lin_net.parameters():
#             l2 = l2 + p.pow(2).sum() #*0.5
#         loss = loss + net.l2_loss * l2

        optim_2.zero_grad()
        loss.backward()
        optim_2.step()
        
        if count%250==0:
            print('[{}] train loss:{:.3f}'.format(count, loss.item()))
            print('[{}] back loss:{:.3f}, mid loss:{:.3f}, front loss:{:.3f}, xy_loss:{:.3f}'.format(count, loss_list[1].item(), loss_list[2].item(), loss_list[3].item(), loss_list[4].item()))


        if count%500==0:
            if encoder_target_type== 'joint':
                train_accuracy = get_accuracy(final_out, y, encoder_target_type, n_digits)
            elif encoder_target_type== "depth_black_white" or encoder_target_type== "depth_black_white_xy_xy":
                accuracy_list = get_accuracy(final_out,y,encoder_target_type, n_digits)
                train_depth_accuracy = accuracy_list[0]
                train_black_accuracy = accuracy_list[1]
                train_white_accuracy = accuracy_list[2]

                print('[{}], train_depth_accuracy:{:.3f}, train_black_accuracy:{:.3f}, train_white_accuracy:{:.3f}'.format(count, train_depth_accuracy, train_black_accuracy, train_white_accuracy))
            elif encoder_target_type== "depth_ordered_one_hot" or encoder_target_type== "depth_ordered_one_hot_xy" :
                if n_digits ==2:
                    accuracy_list = get_accuracy(final_out,y, encoder_target_type, n_digits)
                    train_back_accuracy = accuracy_list[0]
                    train_front_accuracy = accuracy_list[1]
                    
                    edit_loss_list = [x.item() for x in loss_list]
                    
                    count_list.append(count)
                    
                    Tot_loss_list.append(edit_loss_list[0])
                    back_loss_list.append(edit_loss_list[1])
                    front_loss_list.append(edit_loss_list[3])
                    xy_loss_list.append(edit_loss_list[4])
                    
                    back_accuracy_list.append(train_back_accuracy.item())
                    front_accuracy_list.append(train_front_accuracy.item())
                    
                    print('[{}], train_back_accuracy:{:.3f}, train_front_accuracy:{:.3f}'.format(count, train_back_accuracy, train_front_accuracy))

                elif n_digits ==3:
                    accuracy_list = get_accuracy(final_out,y, encoder_target_type , n_digits)
                    train_back_accuracy = accuracy_list[0]
                    train_mid_accuracy = accuracy_list[1]
                    train_front_accuracy = accuracy_list[2]
                    
                    
                    edit_loss_list = [x.item() for x in loss_list]
                    
                    count_list.append(count)
                    
                    Tot_loss_list.append(edit_loss_list[0])
                    back_loss_list.append(edit_loss_list[1])
                    mid_loss_list.append(edit_loss_list[2])
                    front_loss_list.append(edit_loss_list[3])
                    xy_loss_list.append(edit_loss_list[4])
                    
                    back_accuracy_list.append(train_back_accuracy.item())
                    mid_accuracy_list.append(train_mid_accuracy.item())
                    front_accuracy_list.append(train_front_accuracy.item())

                    print('[{}], train_back_accuracy:{:.3f}, train_mid_accuracy:{:.3f}, train_front_accuracy:{:.3f}'.format(count, train_back_accuracy, train_mid_accuracy, train_front_accuracy))

        count +=1 
        if count >= max_iter:
            out = True
            break
            


Lin_model(
  (layer1): Linear(in_features=24, out_features=24, bias=True)
)
1000 iters per epoch
0
[0] train loss:155.390
[0] back loss:9.547, mid loss:0.000, front loss:7.843, xy_loss:138.000
[0], train_back_accuracy:6.000, train_front_accuracy:16.000
[250] train loss:35.041
[250] back loss:4.569, mid loss:0.000, front loss:4.362, xy_loss:26.109
500
[500] train loss:11.051
[500] back loss:3.003, mid loss:0.000, front loss:2.822, xy_loss:5.226
[500], train_back_accuracy:12.000, train_front_accuracy:16.000
[750] train loss:6.096
[750] back loss:2.586, mid loss:0.000, front loss:2.408, xy_loss:1.102
1000
[1000] train loss:4.836
[1000] back loss:2.237, mid loss:0.000, front loss:2.233, xy_loss:0.366
[1000], train_back_accuracy:17.000, train_front_accuracy:15.000
[1250] train loss:4.676
[1250] back loss:2.247, mid loss:0.000, front loss:2.237, xy_loss:0.192
1500
[1500] train loss:4.532
[1500] back loss:2.191, mid loss:0.000, front loss:2.194, xy_loss:0.146
[1500], train_back_accuracy:17.00


count_list = []
Tot_loss_list = []
back_loss_list = []
mid_loss_list = []
front_loss_list = []
xy_loss_list  = []
back_accuracy_list = []
mid_accuracy_list = []
front_accuracy_list = []

In [44]:
output_dir = '/home/riccardo/Desktop/Experiments/LinDecoder/border2/'

fig_lc = plt.figure(figsize = (8,8))
fig_lc.suptitle('Linear Decoder Training \n Min Losses: back: {:.2f}, mid:{:.2f}, front:{:.2f}, xy:{:.2f} '.format(
                   np.min(back_loss_list),
                   np.min(mid_loss_list), np.min(front_loss_list) , 
                    np.min(xy_loss_list) ), fontsize=14)

#plt.figure(figsize = (8,8))
plt.subplot()
plt.plot(count_list, back_loss_list, 'coral', linewidth=2.5, label = "back digit loss")
plt.plot(count_list, mid_loss_list, 'seagreen', linewidth=2.5, label = "mid digit loss")
plt.plot(count_list, front_loss_list, 'dodgerblue', linewidth=2.5, label = "front digit loss")
plt.plot(count_list, xy_loss_list, 'r', linewidth=2.5, label = "location loss")
plt.xlabel("iterations", fontsize=18)
plt.ylabel("Loss", fontsize=18)
#plt.title("losses")
plt.legend(fontsize=15)
plt.grid(True)
plt.savefig('{}/LinDecoder_loss_Curves.png'.format(output_dir))
plt.close()
            

ValueError: zero-size array to reduction operation minimum which has no identity

<Figure size 576x576 with 0 Axes>

In [ ]:
fig_lc = plt.figure(figsize = (8,8))
fig_lc.suptitle('Linear Decoder Training \n Max accuracy: back: {:.2f}, mid:{:.2f}, front:{:.2f} '.format(
                   np.max(back_accuracy_list),
                   np.max(mid_accuracy_list), np.max(front_accuracy_list)),
                   fontsize=20)

#plt.figure(figsize = (8,8))
plt.subplot()
plt.plot(count_list, back_accuracy_list, 'coral', linewidth=2.5, label = "back digit accuracy")
plt.plot(count_list, mid_accuracy_list, 'seagreen', linewidth=2.5, label = "mid digit accuracy")
plt.plot(count_list, front_accuracy_list, 'dodgerblue', linewidth=2.5, label = "front digit accuracy")
plt.xlabel("iterations", fontsize=18)
plt.ylabel("Accuracy", fontsize=18)
#plt.title("losses")
plt.legend(fontsize=12)
plt.grid(True)
plt.savefig('{}/LinDecoder_accuracy_Curves.png'.format(output_dir))
plt.close()
            